<a href="https://colab.research.google.com/github/Walt9819/factores-impacto-desempleo-mexico/blob/main/Python/An%C3%A1lisisExploratorioDENUE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**DENUE**

Análisis exploratorio del conjunto de datos

https://www.inegi.org.mx/app/mapa/denue/default.aspx


### Lectura de archivos

### Filtrado por entidades